In [1]:
import os

# Anzeigen des aktuellen Arbeitsverzeichnisses
current_working_directory = os.getcwd()
print("Aktuelles Arbeitsverzeichnis:", current_working_directory)

Aktuelles Arbeitsverzeichnis: /Users/henrikfliegel/Desktop/Uni/SoSe24/MlClaas/ML-with-CLAAS


In [7]:
import pandas as pd

dfn1 = pd.read_excel('CLAAS_data/S611_Teil_1_P10_20240604.XLSX')
dfn2 = pd.read_excel('CLAAS_data/S611_Teil_2_P10_20240604.XLSX')


In [8]:
def extract_and_create_columns(df, column_name):
    # Überprüfen, ob die Spalte im DataFrame existiert
    if column_name in df.columns:
        # Aufteilen der Spalte in eine Liste von Listen, wobei jede Liste die geteilten Worte enthält
        split_columns = df[column_name].str.split()

        # Erstellen der 'Merkmal' Spalte durch Auswahl des ersten Teils des gesplitteten Strings
        df['Merkmal'] = split_columns.str[0]

        # Erstellen der 'Merkmalwert' Spalte durch Auswahl des zweiten Teils des gesplitteten Strings
        df['Merkmalwert'] = split_columns.str[1]
        
        # Erstellen einer neuen Spalte 'Erste_Drei_Zeichen'
        df['Maschinentyp'] = df['Seriennummer'].str[:3]

    else:
        print(f"Die Spalte {column_name} existiert nicht im DataFrame.")
    return df



In [9]:
dfn = pd.concat([dfn1, dfn2])

df = extract_and_create_columns(dfn, 'Merkmal/Wert/Bezeichng. f. Serienplanun')

merkmale = ['P02', 'N02', 'N08', 'N05', 'B10', 'G02',]

filtered_df = df[df['Merkmal'].isin(merkmale)]

filtered_df_crop = filtered_df.drop(['Statistik-Herkunft', 'Version', 'Monat', 'Woche', 'Periode', 'Werk', 'Material', 'Merkmal/Wert/Bezeichng. f. Serienplanun', 'GeschJahresvariante', 'Zeiger Verw.daten', 'Zeiger Verw.daten', 'Basismengeneinheit', 'Bedarfsmenge'], axis=1).reset_index(drop = True)

filtered_df_crop

,Datum,Seriennummer,CLAAS - Untertyp,Claas-Planungsland,Geschäftsjahr,Bestätigte Menge,Merkmal,Merkmalwert,Maschinentyp
0,2012-11-22,C6613081,820,LTE,2013,1,B10,0040,C66
1,2012-11-22,C6613081,820,LTE,2013,1,G02,0240,C66
2,2012-11-22,C6613081,820,LTE,2013,1,N02,0624,C66
3,2012-11-22,C6613081,820,LTE,2013,1,N05,0121,C66
4,2012-11-22,C6613081,820,LTE,2013,1,N08,0398,C66
...,...,...,...,...,...,...,...,...,...
118397,2024-09-12,C8624511,935,SE,2024,1,G02,0486,C86
118398,2024-09-12,C8624511,935,SE,2024,1,N02,0628,C86
118399,2024-09-12,C8624511,935,SE,2024,1,N05,0424,C86
118400,2024-09-12,C8624511,935,SE,2024,1,N08,0860,C86
